In [ ]:
import wandb

In [ ]:
!wandb login


In [ ]:
import wandb
import os
os.environ['WANDB_DIR'] = os.getcwd() + '/wandb/'
os.environ['WANDB_CACHE_DIR'] = os.getcwd() + '/wandb/.cache/'
os.environ['WANDB_CONFIG_DIR'] = os.getcwd() + '/wandb/.config/'

import torch
import torch.nn as nn
import torch.optim as optim

# 初始化wandb
wandb.init(
    entity='kaifan-li',
    project="my_project"
)

# 构建模型
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(10, 1)
    
    def forward(self, x):
        return self.fc(x)

model = SimpleModel()

# 定义损失函数和优化器
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# 训练循环
for epoch in range(10):
    running_loss = 0.0
    for i in range(100):
        inputs = torch.randn(32, 10)  # 随机生成输入数据
        labels = torch.randn(32, 1)   # 随机生成标签
        
        # 正向传播
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    # 记录训练过程和指标
    avg_loss = running_loss / 100
    wandb.log({"epoch": epoch, "loss": avg_loss})


----